# Imports

In [1]:
from biopandas.pdb import PandasPdb
import pandas as pd
import numpy as np
import os
from copy import deepcopy

In [2]:
os.listdir()

['.ipynb_checkpoints', 'myPDB.pdb', 'Raw_ex1.ipynb', 'receptor.pdb']

# Ouverture du PDB

In [3]:
pdb_receptor = PandasPdb().read_pdb("receptor.pdb")
print(pdb_receptor.df.keys())
pdb_at_df = pdb_receptor.df["ATOM"]

dict_keys(['ATOM', 'HETATM', 'ANISOU', 'OTHERS'])


In [118]:
print(pdb_at_df["element_symbol"].unique())
weigth_d = {
    "C" : 12,
    "N" : 14,
    "O" : 16,
    "S" : 32}
weigth_l = np.array([weigth_d[x] for x in pdb_receptor.df["ATOM"]["element_symbol"]])

['N' 'C' 'O' 'S']


## COM non pondéré

In [30]:
np_coords = pdb_at_df[["x_coord", "y_coord", "z_coord"]].values
COM_prot = np.average(np_coords, axis = 0)
COM_prot

array([31.01180928,  1.09121847,  5.2611775 ])

## COM pondérée

# COM par résidu

Isole les carbones alphas de la structure d'origine, pour accueillir les COM des résidus

In [15]:
res_CA = pdb_at_df[pdb_at_df["atom_name"] == "CA"]
res_CA.head()

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
1,ATOM,2,,CA,,LEU,,G,67,,...,36.029,29.143,6.093,1.0,0.0,,AP1,C,NaN,1
9,ATOM,10,,CA,,ARG,,G,68,,...,34.269,30.180,2.853,1.0,0.0,,AP1,C,NaN,9
20,ATOM,21,,CA,,ALA,,G,69,,...,32.997,26.981,1.019,1.0,0.0,,AP1,C,NaN,20
25,ATOM,26,,CA,,HIS,,G,70,,...,34.316,23.593,-0.151,1.0,0.0,,AP1,C,NaN,25
35,ATOM,36,,CA,,ILE,,G,71,,...,33.208,20.980,-2.692,1.0,0.0,,AP1,C,NaN,35


Calcul du COM de chaques résidus

In [16]:
COM_res = pdb_at_df.groupby(["chain_id", "residue_number"], as_index=False)[["x_coord", "y_coord", "z_coord"]].mean()
COM_res.head()

,chain_id,residue_number,x_coord,y_coord,z_coord
0,G,67,35.853125,30.109625,7.08525
1,G,68,34.735091,31.197273,0.27200
2,G,69,33.637200,26.896200,0.74380
3,G,70,35.520700,22.442900,0.09140
4,G,71,33.097750,21.313750,-3.59350


In [18]:
res_CA[["x_coord","y_coord","z_coord"]] = COM_res[["x_coord","y_coord","z_coord"]].values
res_CA

C:\Users\dom\Anaconda3\envs\PD_inria\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\dom\Anaconda3\envs\PD_inria\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
1,ATOM,2,,CA,,LEU,,G,67,,...,35.853125,30.109625,7.085250,1.0,0.0,,AP1,C,NaN,1
9,ATOM,10,,CA,,ARG,,G,68,,...,34.735091,31.197273,0.272000,1.0,0.0,,AP1,C,NaN,9
20,ATOM,21,,CA,,ALA,,G,69,,...,33.637200,26.896200,0.743800,1.0,0.0,,AP1,C,NaN,20
25,ATOM,26,,CA,,HIS,,G,70,,...,35.520700,22.442900,0.091400,1.0,0.0,,AP1,C,NaN,25
35,ATOM,36,,CA,,ILE,,G,71,,...,33.097750,21.313750,-3.593500,1.0,0.0,,AP1,C,NaN,35
43,ATOM,44,,CA,,ASP,,G,72,,...,34.586000,17.296375,-1.443500,1.0,0.0,,AP1,C,NaN,43
51,ATOM,52,,CA,,TYR,,G,73,,...,33.552583,13.696583,-2.743667,1.0,0.0,,AP1,C,NaN,51
63,ATOM,64,,CA,,LEU,,G,74,,...,30.868625,15.430500,-8.951750,1.0,0.0,,AP1,C,NaN,63
71,ATOM,72,,CA,,ARG,,G,75,,...,29.942364,9.639182,-9.546273,1.0,0.0,,AP1,C,NaN,71
82,ATOM,83,,CA,,LEU,,G,76,,...,28.004375,13.415750,-13.582750,1.0,0.0,,AP1,C,NaN,82


In [21]:
tmp = deepcopy(pdb_receptor)
tmp.df["ATOM"] = res_CA

{'ATOM':      record_name  atom_number blank_1 atom_name alt_loc residue_name blank_2  \
 1           ATOM            2                CA                  LEU           
 9           ATOM           10                CA                  ARG           
 20          ATOM           21                CA                  ALA           
 25          ATOM           26                CA                  HIS           
 35          ATOM           36                CA                  ILE           
 43          ATOM           44                CA                  ASP           
 51          ATOM           52                CA                  TYR           
 63          ATOM           64                CA                  LEU           
 71          ATOM           72                CA                  ARG           
 82          ATOM           83                CA                  LEU           
 90          ATOM           91                CA                  THR           
 97          ATOM   

In [20]:
PandasPdb.to_pdb(tmp, "myPDB.pdb")

# PLOT

In [68]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import numpy as np
init_notebook_mode(connected=True)

In [41]:
all_coords = pdb_receptor.df["ATOM"][["x_coord","y_coord","z_coord"]].values
COMres_coords = tmp.df["ATOM"][["x_coord","y_coord","z_coord"]].values
# COM_prot

In [42]:
all_coords, all_coords[:,0]

(array([[ 37.475,  29.26 ,   6.238],
        [ 36.029,  29.143,   6.093],
        [ 35.44 ,  30.053,   5.019],
        ...,
        [ 32.496, -26.301,  16.35 ],
        [ 32.407, -24.779,  16.167],
        [ 31.267, -26.806,  17.124]]),
 array([37.475, 36.029, 35.44 , ..., 32.496, 32.407, 31.267]))

In [50]:
trace1 = go.Scatter3d(x=all_coords[:,0],y=all_coords[:,1],z=all_coords[:,2],
    mode='markers',
    marker=dict(size=1, line=dict(color='rgba(0,0,0, 0.14)', width=0.5), opacity=0.8))
trace2 = go.Scatter3d(x=COMres_coords[:,0],y=COMres_coords[:,1],z=COMres_coords[:,2],
    mode='markers',
    marker=dict(size=4, line=dict(color='rgba(217, 217, 217, 0.14)', width=0.5), opacity=0.8))
trace3 = go.Scatter3d(x=[COM_prot[0]],y=[COM_prot[1]],z=[COM_prot[2]],
    mode='markers',
    marker=dict(size=12, line=dict(color='rgba(255, 0, 0, 0.75)', width=0.5), opacity=0.8))

data = [trace1, trace2, trace3]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))

fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Rg

In [128]:
def get_rg(coor, weight = np.array([])):
    """
    coor = 3D numpy array
    """
    if weight.size == 0:
        print("####")
        weight = np.ones(coor.shape[0])
    weight_sum = weight.sum()
        
    COM = np.average(coor, axis = 0, weights=weight)
    weighted_pos = (weight * coor.T).T
    rg = np.sqrt(((np.square(weighted_pos - COM).T * weight).T).sum() / weight_sum)
    return (rg)
        
        
get_rg(all_coords, weigth_l)

543.679808839006

In [100]:
o = np.array([[1,2,3],[4,5,6]])
m = np.array([1,2])
(m*o.T).T

array([[ 1,  2,  3],
       [ 8, 10, 12]])